In [ ]:
# Importa o módulo 'csv' da biblioteca padrão do Python, usado para manipular arquivos CSV.
import csv

# Define o caminho completo do arquivo CSV a ser lido.
caminho_arquivo = r'C:\Users\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025.csv'

# Abre o arquivo em modo leitura ('r'), com codificação UTF-8
with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
    # Cria um objeto Sniffer, que detecta automaticamente o formato (dialeto) do CSV
    sniffer = csv.Sniffer()

    # Lê uma amostra inicial do arquivo (2048 caracteres) para que o Sniffer possa analisar
    amostra = arquivo.read(2048)

    # Retorna o ponteiro para o início do arquivo, pois a leitura anterior avançou o cursor
    arquivo.seek(0)

    # Detecta automaticamente o delimitador (como vírgula, ponto e vírgula, etc.) e o caractere de aspas
    dialeto = sniffer.sniff(amostra)

    # Exibe informações sobre o dialeto detectado
    print(f"Delimitador detectado: {repr(dialeto.delimiter)}")
    print(f"Caractere de aspas detectado: {repr(dialeto.quotechar)}")
    print(f"Primeiras 5 linhas para inspeção:")

    # Cria um leitor CSV usando o dialeto detectado
    leitor = csv.reader(arquivo, delimiter=dialeto.delimiter, quotechar=dialeto.quotechar)

    # Itera pelas primeiras 5 linhas do arquivo e imprime cada uma
    for i, linha in enumerate(leitor):
        print(linha)
        if i >= 4:
            break  # Interrompe o loop após imprimir 5 linhas

In [ ]:
import csv

# Aumenta o limite de caracteres permitidos por campo no CSV para 10 milhões.
# Isso evita erros quando campos (como "inteiro_teor") têm textos muito longos.
csv.field_size_limit(10_000_000)

# Define o caminho do arquivo CSV que será lido
caminho_arquivo = r'C:\Users\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025.csv'

# Abre o arquivo em modo leitura, com codificação UTF-8
with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
    
    # Cria um leitor CSV, assumindo que os campos são separados por tabulação (\t)
    # e que os campos podem estar entre aspas duplas
    leitor = csv.reader(arquivo, delimiter='\t', quotechar='"')
    
    # Lê a primeira linha do arquivo como cabeçalho
    cabecalho = next(leitor)
    
    # Conta quantas colunas existem no cabeçalho (quantidade esperada por linha)
    num_colunas = len(cabecalho)

    # Inicializa uma lista para armazenar as linhas que têm problemas
    linhas_problematicas = []

    # Percorre cada linha do CSV, a partir da segunda (por isso start=2)
    for i, linha in enumerate(leitor, start=2):
        # Verifica se o número de colunas da linha atual bate com o do cabeçalho
        if len(linha) != num_colunas:
            # Se não bater, adiciona à lista de linhas problemáticas
            linhas_problematicas.append((i, linha))

# Exibe quantas linhas foram identificadas com erro de coluna
print(f"{len(linhas_problematicas)} linhas com número incorreto de colunas.")

# Exibe as primeiras 5 linhas problemáticas para inspeção
for linha in linhas_problematicas[:5]:
    print(f"Linha {linha[0]}: {linha[1]}")

In [ ]:
import pandas as pd  

# Define o caminho local do arquivo CSV a ser lido
caminho_arquivo = r'C:\Users\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025.csv'

# Lê o arquivo CSV utilizando separador por tabulação, tratando eventuais problemas de parsing
df = pd.read_csv(
    caminho_arquivo,
    sep='\t',            # Usa tabulação como separador (arquivo TSV)
    quotechar='"',       # Define o caractere de aspas para lidar com campos que contêm separadores internos
    encoding='utf-8',    # Define a codificação do arquivo
    on_bad_lines='warn', # Emite um aviso quando encontra linhas com erro de estrutura (como número incorreto de colunas)
    engine='python'      # Usa o motor Python, mais tolerante a campos com quebras de linha internas
)

# Exibe os nomes das colunas identificadas no DataFrame
print(df.columns)

# Mostra o tamanho da tabela (número de linhas e colunas)
print(df.shape)

# Exibe as duas primeiras linhas da tabela transpostas (útil para visualizar os valores de forma vertical por campo)
print(df.head(2).transpose())

In [ ]:
# Caminhos para o arquivo de entrada (original) e o arquivo de saída (limpo)
input_path = r'C:\Users\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025.csv'
output_path = r'C:\Users\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025_V1.csv'

linhas_validas = []  # Lista onde serão armazenadas apenas as linhas válidas

# Abre o arquivo CSV original para leitura
with open(input_path, 'r', encoding='utf-8') as f_in:
    leitor = csv.reader(f_in, delimiter='\t', quotechar='"')  # Usa tabulação como separador e aspas como delimitador de campo
    for row in leitor:
        # Ignora linhas que contenham um marcador de fim customizado
        if len(row) == 1 and row[0].startswith('#####fim#####id:'):
            continue
        # Adiciona à lista apenas linhas que têm exatamente 13 colunas (esperado no esquema)
        if len(row) == 13:
            linhas_validas.append(row)

# Abre (ou cria) um novo arquivo para salvar as linhas limpas
with open(output_path, 'w', encoding='utf-8', newline='') as f_out:
    escritor = csv.writer(f_out, delimiter='\t', quotechar='"', quoting=csv.QUOTE_ALL)  # Define o formato de escrita
    # Escreve o cabeçalho no novo arquivo (nomes das 13 colunas esperadas)
    escritor.writerow([
        'numero_processo', 'comarca', 'serventia', 'assunto', 'classe',
        'polo_ativo', 'polo_passivo', 'cpf_cnpj_polo_ativo', 'cpf_cnpj_polo_passivo',
        'inteiro_teor', 'codg_cnj_audi', 'codg_cnj_julgamento', 'classificacao'
    ])
    # Escreve todas as linhas válidas coletadas anteriormente
    escritor.writerows(linhas_validas)

# Exibe quantas linhas válidas foram salvas no novo arquivo
print(f"{len(linhas_validas)} registros válidos salvos.")

4997 registros válidos salvos.


In [108]:
try:
    # Tenta ler o CSV limpo (salvo anteriormente) usando o pandas
    df = pd.read_csv(
        output_path,        # Caminho para o arquivo já limpo
        sep='\t',           # Delimitador de campos: tabulação
        quotechar='"',      # Caractere que envolve textos com separadores
        encoding='utf-8',   # Codificação do arquivo
        engine='python'     # Usa o engine "python" para lidar melhor com campos complexos
    )
except Exception as e:
    # Captura e exibe qualquer erro ocorrido na leitura do arquivo
    print(f"Erro ao carregar CSV: {e}")

# Exibe o número de linhas e colunas do DataFrame carregado
print(df.shape)

# Mostra as duas primeiras linhas transpostas (útil para inspeção vertical das colunas)
print(df.head(2).transpose())

(4997, 13)
                                           0  \
numero_processo              numero_processo   
comarca                              comarca   
serventia                          serventia   
assunto                              assunto   
classe                                classe   
polo_ativo                        polo_ativo   
polo_passivo                    polo_passivo   
cpf_cnpj_polo_ativo      cpf_cnpj_polo_ativo   
cpf_cnpj_polo_passivo  cpf_cnpj_polo_passivo   
inteiro_teor                    inteiro_teor   
codg_cnj_audi                  codg_cnj_audi   
codg_cnj_julgamento      codg_cnj_julgamento   
classificacao                  classificacao   

                                                                       1  
numero_processo                                5000399.26.2025.8.09.0103  
comarca                                                           MINAÇU  
serventia                            Minaçu - Vara das Fazendas Públicas  
assunto         

In [109]:
# Conta quantas vezes cada valor aparece na coluna 'classificacao'
# Útil para ver o balanço entre classes como 'frutifero' e 'infrutifero'
df['classificacao'].value_counts()

# Calcula a proporção de valores ausentes (NaN) em cada coluna
# Mostra em ordem decrescente de colunas mais incompletas para menos
df.isna().mean().sort_values(ascending=False)

# Mede o comprimento (número de caracteres) de cada valor na coluna 'inteiro_teor'
# Em seguida, gera estatísticas descritivas (média, mínimo, máximo, quartis etc.)
df['inteiro_teor'].str.len().describe()

count      4997.000000
mean      14294.346008
std       23438.525962
min          12.000000
25%        5634.000000
50%        9685.000000
75%       17205.000000
max      813597.000000
Name: inteiro_teor, dtype: float64

In [110]:
import pandas as pd

# Função para gerar um resumo estatístico e informativo de um DataFrame
def resumo_dataset(df):
    resumo = pd.DataFrame({
        # Tipo de dado de cada coluna (object, int64, float64, etc.)
        'Tipo de Dado': df.dtypes,
        
        # Total de linhas (será o mesmo para todas as colunas)
        'Total de Valores': len(df),
        
        # Quantidade de valores não nulos por coluna
        'Não Nulos': df.notnull().sum(),
        
        # Quantidade de valores nulos por coluna
        'Nulos': df.isnull().sum(),
        
        # Porcentagem de valores nulos por coluna
        '% de Nulos': df.isnull().mean() * 100,
        
        # Número de valores únicos por coluna
        'Valores Únicos': df.nunique(),
        
        # Um exemplo aleatório de valor não nulo da coluna (útil para inspecionar)
        'Exemplo': df.apply(lambda x: x.dropna().sample(1).values[0] if x.dropna().size > 0 else None)
    })

    # Reorganiza as colunas para melhor leitura
    resumo = resumo[['Tipo de Dado', 'Total de Valores', 'Não Nulos', 'Nulos', '% de Nulos', 'Valores Únicos', 'Exemplo']]
    
    # Ordena pelas colunas com maior percentual de nulos no topo
    return resumo.sort_values(by='% de Nulos', ascending=False)

# Aplica a função ao DataFrame carregado
resumo_df = resumo_dataset(df)

# Exibe o resultado de forma legível no notebook
from IPython.display import display
display(resumo_df)

,Tipo de Dado,Total de Valores,Não Nulos,Nulos,% de Nulos,Valores Únicos,Exemplo
codg_cnj_julgamento,object,4997,2618,2379,47.608565,5,466
codg_cnj_audi,object,4997,3828,1169,23.394036,10,12624
polo_passivo,object,4997,4951,46,0.920552,3711,Instituto Nacional Do Seguro Social
cpf_cnpj_polo_passivo,object,4997,4951,46,0.920552,3536,59285411000113
numero_processo,object,4997,4997,0,0.000000,4997,5103957.92.2025.8.09.0077
classe,object,4997,4997,0,0.000000,77,PROCESSO CÍVEL E DO TRABALHO -> Processo de Co...
assunto,object,4997,4997,0,0.000000,493,DIREITO CIVIL -> Obrigações -> Espécies de Tít...
serventia,object,4997,4997,0,0.000000,430,"Goiânia - 1ª UPJ Varas Cíveis: 1ª, 2ª, 3ª, 4ª ..."
comarca,object,4997,4997,0,0.000000,117,GOIÂNIA
cpf_cnpj_polo_ativo,object,4997,4997,0,0.000000,3724,71497431107


In [111]:
# Conta a frequência de cada valor presente na coluna 'classificacao'
# O parâmetro dropna=False garante que valores ausentes (NaN) também sejam contabilizados
df['classificacao'].value_counts(dropna=False)

classificacao
frutifero        2617
infrutifero      2379
classificacao       1
Name: count, dtype: int64

In [112]:
df[df['classificacao'] == 'classificacao']

,numero_processo,comarca,serventia,assunto,classe,polo_ativo,polo_passivo,cpf_cnpj_polo_ativo,cpf_cnpj_polo_passivo,inteiro_teor,codg_cnj_audi,codg_cnj_julgamento,classificacao
0,numero_processo,comarca,serventia,assunto,classe,polo_ativo,polo_passivo,cpf_cnpj_polo_ativo,cpf_cnpj_polo_passivo,inteiro_teor,codg_cnj_audi,codg_cnj_julgamento,classificacao


In [113]:
# Conta quantas linhas duplicadas existem no DataFrame inteiro (todas as colunas devem ser idênticas)
print(df.duplicated().sum())

# Remove as linhas duplicadas, mantendo apenas a primeira ocorrência
df = df.drop_duplicates()

0


In [114]:
# Converte a coluna 'codg_cnj_julgamento' para tipo numérico (float), 
# forçando valores inválidos (como strings não numéricas) a serem convertidos em NaN
df['codg_cnj_julgamento'] = pd.to_numeric(df['codg_cnj_julgamento'], errors='coerce')

# Faz o mesmo para a coluna 'codg_cnj_audi', garantindo que ambas tenham formato numérico consistente
df['codg_cnj_audi'] = pd.to_numeric(df['codg_cnj_audi'], errors='coerce')

In [ ]:
# Remove espaços em branco no início/fim dos nomes das colunas (evita bugs por nomes mal formatados)
df.columns = df.columns.str.strip()

# Aplica o mesmo tratamento para todos os valores do DataFrame que forem strings: remove espaços extras
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [116]:
# Substitui quebras de linha (\n ou \r) no campo 'inteiro_teor' por um espaço simples
# Isso garante que o texto fique em uma linha contínua, facilitando análise e visualização
df['inteiro_teor'] = df['inteiro_teor'].str.replace(r'\n|\r', ' ', regex=True)

In [ ]:
# Exporta o DataFrame limpo para um novo arquivo CSV
# - 'sep="\t"' define que o separador será tabulação (TSV)
# - 'index=False' evita salvar o índice numérico do pandas como uma coluna no CSV
# - 'encoding="utf-8"' assegura compatibilidade com acentuação e caracteres especiais
df.to_csv(r'C:\Users\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025_V2.csv', sep='\t', index=False, encoding='utf-8')

In [118]:
def validar_e_corrigir_linha(linha, colunas_esperadas):
    """
    Valida e corrige uma linha de CSV.
    Retorna a linha se for válida, ou None se for inválida.

    Parâmetros:
    - linha: list[str] - linha bruta lida do arquivo
    - colunas_esperadas: int | list[str] - número esperado de colunas (ou lista de nomes de coluna)

    Regras de validação aplicadas:
    """

    # 1. Elimina linhas que são marcadores de fim de lote/exportação,
    #    normalmente usadas por sistemas para indicar encerramento de bloco
    if len(linha) == 1 and linha[0].startswith('#####fim#####id:'):
        return None

    # 2. Elimina linhas que repetem o cabeçalho original (pode ocorrer em arquivos concatenados),
    #    comparando diretamente com a lista de nomes de colunas
    if linha == colunas_esperadas:
        return None

    # 3. Descarta linhas com número incorreto de colunas (pode indicar quebra de linha mal formatada ou dado incompleto)
    if len(linha) != len(colunas_esperadas):
        return None

    # 4. Aplica strip() em cada campo textual para remover espaços em branco desnecessários
    linha = [campo.strip() if isinstance(campo, str) else campo for campo in linha]

    # Retorna a linha limpa se passou por todas as validações
    return linha

In [ ]:
import csv

# Caminhos dos arquivos de entrada (original) e saída (limpo)
input_path = r'C:\Users\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025_V2.csv'
output_path = r'C:\Users\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025_V3.csv'

# Lista de nomes de colunas esperadas no CSV, usada tanto como cabeçalho quanto para validação
colunas = [
    'numero_processo', 'comarca', 'serventia', 'assunto', 'classe',
    'polo_ativo', 'polo_passivo', 'cpf_cnpj_polo_ativo', 'cpf_cnpj_polo_passivo',
    'inteiro_teor', 'codg_cnj_audi', 'codg_cnj_julgamento', 'classificacao'
]

# Função que valida e corrige uma linha do CSV, aplicando regras específicas
def validar_e_corrigir_linha(linha, colunas_esperadas):
    # Ignora linhas que contêm apenas um campo especial de finalização
    if len(linha) == 1 and linha[0].startswith('#####fim#####id:'):
        return None
    # Ignora linhas que repetem o cabeçalho (ocorre em arquivos concatenados)
    if linha == colunas_esperadas:
        return None
    # Ignora linhas com número incorreto de colunas
    if len(linha) != len(colunas_esperadas):
        return None
    # Aplica strip() a cada campo textual para limpeza
    return [campo.strip() if isinstance(campo, str) else campo for campo in linha]

# Lista para armazenar apenas as linhas válidas
linhas_validas = []

# Leitura do arquivo original linha a linha, aplicando validação
with open(input_path, 'r', encoding='utf-8') as f_in:
    leitor = csv.reader(f_in, delimiter='\t', quotechar='"')
    for linha in leitor:
        nova_linha = validar_e_corrigir_linha(linha, colunas)
        if nova_linha:
            linhas_validas.append(nova_linha)

# Escrita do novo arquivo com as linhas válidas e cabeçalho padronizado
with open(output_path, 'w', encoding='utf-8', newline='') as f_out:
    escritor = csv.writer(f_out, delimiter='\t', quotechar='"', quoting=csv.QUOTE_ALL)
    escritor.writerow(colunas)           # Escreve o cabeçalho
    escritor.writerows(linhas_validas)   # Escreve todas as linhas válidas

# Mensagem de confirmação com total de registros válidos salvos
print(f"{len(linhas_validas)} linhas válidas salvas em {output_path}")

In [ ]:
# Define o caminho para o CSV já limpo (pré-processado) e o caminho de saída do JSON
csv_limpado = r'C:\Users\anonymous\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025_V3.csv'
json_saida = r'C:\Users\anonymous\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025.json'

# Lê o arquivo CSV usando tabulação como delimitador e aspas duplas como delimitador de texto
df = pd.read_csv(csv_limpado, sep='\t', quotechar='"', encoding='utf-8')

# Exporta para JSONL (JSON por linha) com codificação preservando acentuação (force_ascii=False)
# 'orient=records' exporta cada linha como um dicionário separado (lista de registros)
# 'lines=True' escreve cada dicionário em uma linha do arquivo => formato JSONL
df.to_json(json_saida, orient='records', force_ascii=False, lines=True)

# Confirmação visual para o usuário
print(f"Arquivo JSON exportado com sucesso para: {json_saida}")

In [ ]:
# Caminhos de entrada e saída
input_json_path = r"C:\Users\anonymous\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025.json"
output_json_path = r"C:\Users\anonymous\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025_V1.json"

In [124]:
import re
import pandas as pd

# Remove marcadores artificiais do texto que indicam início ou fim de registros
def limpar_marcadores(texto):
    return re.sub(r'>>>>>inicio<<<<<|#####fim#####id:\d+', '', texto)

# Remove URLs do texto (http ou https seguido por qualquer caractere não-espaço)
def remover_links(texto):
    return re.sub(r'https?://\S+', '', texto)

# Remove trechos que contenham palavras-chave relacionadas a assinaturas e autenticações
def remover_assinaturas(texto):
    return re.sub(
        r'(assinado|assinatura|OAB|Matricula|Hash SHA256|autenticação eletrônica).*',
        '', texto, flags=re.IGNORECASE
    )

# Normaliza os espaços: substitui múltiplos espaços, quebras de linha e tabs por um único espaço
def normalizar_espacos(texto):
    return re.sub(r'\s+', ' ', texto).strip()

# Anonimiza dados pessoais como CPF, RG e endereços com padrão "Rua ..."
def anonimizar_dados_pessoais(texto):
    texto = re.sub(r'\b\d{3}\.?\d{3}\.?\d{3}-?\d{2}\b', '[CPF]', texto)  # CPF
    texto = re.sub(r'\b\d{2}\.?\d{3}\.?\d{3}-?\d{1,2}\b', '[RG]', texto)   # RG
    texto = re.sub(r'Rua [^\.,;\n]+', 'Rua [REMOVIDO]', texto, flags=re.IGNORECASE)  # Endereço
    return texto

# Função principal: encadeia todas as funções de limpeza acima
def limpar_inteiro_teor(texto):
    if not isinstance(texto, str):  # Garante que só aplica em strings
        return texto
    texto = limpar_marcadores(texto)
    texto = remover_links(texto)
    texto = remover_assinaturas(texto)
    texto = anonimizar_dados_pessoais(texto)
    texto = normalizar_espacos(texto)
    return texto

In [ ]:
# Carrega o JSONL (JSON Lines), onde cada linha representa um dicionário separado
df = pd.read_json(input_json_path, lines=True, encoding='utf-8')

# Aplica a função de limpeza 'limpar_inteiro_teor' (definida anteriormente) em cada valor da coluna 'inteiro_teor'
df['inteiro_teor'] = df['inteiro_teor'].apply(limpar_inteiro_teor)

# Exporta o DataFrame novamente para JSONL, com uma linha por registro (orient='records', lines=True)
# force_ascii=False garante que caracteres acentuados sejam mantidos corretamente no arquivo
df.to_json(output_json_path, orient='records', lines=True, force_ascii=False)

# Mensagem de confirmação
print(f"Arquivo JSON limpo salvo com sucesso em:\n{output_json_path}")

In [ ]:
import pandas as pd

# Caminho para o arquivo CSV (mesmo que tenha extensão .json, trate como CSV!)
caminho_arquivo = r"C:\Users\anonymous\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025_V1.json"

# Tenta abrir como CSV tabulado
df = pd.read_csv(caminho_arquivo, sep='\t', quoting=3, encoding='utf-8', on_bad_lines='skip')

# Verifica colunas
print("Colunas detectadas:")
print(df.columns)

# Mostra primeiras linhas
print(df.head())

Colunas detectadas:
Index(['{"numero_processo":"numero_processo","comarca":"comarca","serventia":"serventia","assunto":"assunto","classe":"classe","polo_ativo":"polo_ativo","polo_passivo":"polo_passivo","cpf_cnpj_polo_ativo":"cpf_cnpj_polo_ativo","cpf_cnpj_polo_passivo":"cpf_cnpj_polo_passivo","inteiro_teor":"inteiro_teor","codg_cnj_audi":null,"codg_cnj_julgamento":null,"classificacao":"classificacao"}'], dtype='object')
  {"numero_processo":"numero_processo","comarca":"comarca","serventia":"serventia","assunto":"assunto","classe":"classe","polo_ativo":"polo_ativo","polo_passivo":"polo_passivo","cpf_cnpj_polo_ativo":"cpf_cnpj_polo_ativo","cpf_cnpj_polo_passivo":"cpf_cnpj_polo_passivo","inteiro_teor":"inteiro_teor","codg_cnj_audi":null,"codg_cnj_julgamento":null,"classificacao":"classificacao"}
0  {"numero_processo":"5000399.26.2025.8.09.0103"...                                                                                                                                               

In [ ]:
import pandas as pd

# Caminho do arquivo
caminho_arquivo = r"C:\Users\anonymous\Documents\Modelos\Perfil-Conciliação-Xgboost-GCP-V2\dados\anonymous_03072025\dados_processos_anonymous_03072025_V1.json"

# Leitura do JSON
df = pd.read_json(caminho_arquivo, lines=True)

# Exibe o número de linhas (registros) e colunas (chaves por registro)
print("Número de linhas (registros):", len(df))
print("Número de colunas:", len(df.columns))

# (Opcional) Exibe os nomes das colunas
print("\nColunas:")
print(df.columns.tolist())

Número de linhas (registros): 4997
Número de colunas: 13

Colunas:
['numero_processo', 'comarca', 'serventia', 'assunto', 'classe', 'polo_ativo', 'polo_passivo', 'cpf_cnpj_polo_ativo', 'cpf_cnpj_polo_passivo', 'inteiro_teor', 'codg_cnj_audi', 'codg_cnj_julgamento', 'classificacao']


In [ ]:
pip install tiktoken

In [129]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
df['tokens_gpt'] = df['inteiro_teor'].astype(str).apply(lambda x: len(encoding.encode(x)))
print(df['tokens_gpt'].describe())

count     4997.000000
mean      1728.011607
std       3062.886766
min          0.000000
25%        206.000000
50%       1139.000000
75%       2297.000000
max      57504.000000
Name: tokens_gpt, dtype: float64


In [130]:
df['tokens_gpt'].value_counts().sort_index()

tokens_gpt
0        81
1         1
3        21
4        13
5        37
         ..
42269     1
43074     1
43181     1
57409     1
57504     1
Name: count, Length: 2582, dtype: int64

In [131]:
bins = [0, 2000, 4000, 8000, 16000, float('inf')]
labels = ['até 2k', '2k–4k', '4k–8k', '8k–16k', '+16k']
df['faixa_tokens'] = pd.cut(df['tokens_gpt'], bins=bins, labels=labels)

df['faixa_tokens'].value_counts().sort_index()

faixa_tokens
até 2k    3389
2k–4k     1047
4k–8k      416
8k–16k      42
+16k        22
Name: count, dtype: int64